In [139]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.cluster import DBSCAN 

# import the vectorized data
df = pd.read_csv("chp5_classifiers/vector_data.csv")
df.head()

,language,cast_0,director,genre_0,company_0,country,cast_num,crew_num,popularity
0,0,0,0,0,0,0,3,3,1.0
1,0,1,1,1,1,0,1,2,1.0
2,0,2,2,0,2,1,3,3,1.0
3,0,3,3,0,3,0,3,3,1.0
4,0,4,4,0,1,0,1,3,1.0


In [145]:
pop = len([x for x in df['popularity'] if x == 1.0])
print("popular: ", pop)
print("not popular: ", len(df) - pop)

popular:  2393
not popular:  2397


# Compute Self-defined Metrics

## nominal and ordinal attributes

We have both nominal and ordinal attributes in the vectors.<br>
Similarity:
1. for nominal attribute, s = 1 if p == q, s = 0 if p != q
2. for ordinal attribute, s = 1 - abs(p - q)/(n - 1)
3. n: range of the mapping, in our case, n = 4
<br> 
This takes much longer time to compute. The difference is very little. In consideration of simplicity, we choose to use <b>SMC</b> to compute the metrics.

In [48]:
# for developing purpose
df = df.head(200)

columns = ['language', 'cast_0', 'director', 'genre_0', 'company_0', 'country', 'cast_num', 'crew_num']
n = 4

# similarity matrix
simatrix = np.zeros((len(df), len(df)))
# distance matrix
dismatrix = np.zeros((len(df), len(df)))

def metrics(df):
    # for each row combination
    for i in range(len(df) - 1):
        for j in range(i + 1, len(df)):
            # similarity vector of p and q
            s = []
            # nominal
            for col in columns[:6]:
                p = df.loc[i, col]
                q = df.loc[j, col]
                if p == q:
                    s.append(1)
                else:
                    s.append(0)
            # ordinal
            for col in columns[6:]:
                p = df.loc[i, col]
                q = df.loc[j, col]
                s.append(1 - abs(p - q)/(n - 1))
            # compute magnitude of the similarity vector, normalize
            mag = np.linalg.norm(s)/math.sqrt(8)
            # compute magnitude of the disimilarity vector, normalize
            magd = np.linalg.norm([(1 - x) for x in s ])/math.sqrt(8)
            # insert into similarity matrix
            simatrix[i][j] = mag
            simatrix[j][i] = mag
            # insert into dismatrix
            dismatrix[i][j] = magd
            dismatrix[j][i] = magd
        simatrix[i][i] = 1
        dismatrix[i][i] = 0
    

# compute and export the similarity matrix into a csv file
metrics(df)
pd.DataFrame(simatrix).to_csv('output/simatrix_200.csv', index=False)
pd.DataFrame(dismatrix).to_csv('output/dismatrix_200.csv', index=False)

## SMC to compute metrics
treat all attributes as nominal attributes. Use SMC to do binary computation of similarity and disimilarity (distance)

In [141]:
# for testing purpose
# df = df.head(200)

columns = ['language', 'cast_0', 'director', 'genre_0', 'company_0', 'country', 'cast_num', 'crew_num']

# similarity matrix
simatrix = np.zeros((len(df), len(df)))
# distance matrix
dismatrix = np.zeros((len(df), len(df)))

# simpler metrics computing method that only computes as nominal attribute
def metrics_simple(df):
    # for each row combination
    for i in range(len(df) - 1):
        for j in range(i + 1, len(df)):
            # similarity vector of p and q
            s = 0
            # nominal
            for col in columns:
                p = df.loc[i, col]
                q = df.loc[j, col]
                if p == q:
                    s += 1
            # insert into similarity matrix
            simatrix[i][j] = s/8
            simatrix[j][i] = s/8
            # insert into dismatrix
            dismatrix[i][j] = (8-s)/8
            dismatrix[j][i] = (8-s)/8
        simatrix[i][i] = 1
        dismatrix[i][i] = 0
        if i%100 == 0:
            print(i, end = '...')

# compute and export the similarity matrix into a csv file
metrics_simple(df)
pd.DataFrame(simatrix).to_csv('output/simatrix.csv', index=False)
pd.DataFrame(dismatrix).to_csv('output/dismatrix.csv', index=False)

0...100...200...300...400...500...600...700...800...900...1000...1100...1200...1300...1400...1500...1600...1700...1800...1900...2000...2100...2200...2300...2400...2500...2600...2700...2800...2900...3000...3100...3200...3300...3400...3500...3600...3700...3800...3900...4000...4100...4200...4300...4400...4500...4600...4700...

# Implement Clustering Algorithms

## 1. DBScan

1. How to determine eps and min_samples?<br>
A low minPts means it will build more clusters from noise. One heuristic approach is use ln(n), where n is the total number of points to be clustered.
2. To determine elipson:<br>
<b>k-distance plot</b> <br>
In a clustering with minPts = k, we expect that core pints and border points' k-distance are within a certain range, while noise points can have much greater k-distance, thus we can observe a knee point in the k-distance plot. However, sometimes there may be no obvious knee, or there can be multiple knees, which makes it hard to decide

In [62]:
print(len(dismatrix))
np.log(4790)

4790


8.474285690404962

In [143]:
# use disimatrix (distance) computed by nominal
dbscan = DBSCAN(eps=0.374, min_samples=9, metric="precomputed").fit_predict(dismatrix)
print('eps=0.374\n', dbscan[:100])
print('number of clusters: ', max(dbscan) + 1)
print('number of noises: ', len([x for x in dbscan if x == -1]))

dbscan = DBSCAN(eps=0.375, min_samples=9, metric="precomputed").fit_predict(dismatrix)
print('eps=0.375\n', dbscan[:100])
print('number of clusters: ', max(dbscan) + 1)
print('number of noises: ', len([x for x in dbscan if x == -1]))

dbscan = DBSCAN(eps=0.38, min_samples=9, metric="precomputed").fit_predict(dismatrix)
print('eps=0.38\n', dbscan[:100])
print('number of clusters: ', max(dbscan) + 1)
print('number of noises: ', len([x for x in dbscan if x == -1]))

eps=0.374
 [-1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1
 -1 -1 -1 -1  1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
number of clusters:  13
number of noises:  4015
eps=0.375
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0 -1  0  0 -1  0
 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0
  0 -1 -1  0]
number of clusters:  1
number of noises:  639
eps=0.38
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0 -1  0  0 -1  0
 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

As observed above, values after 0.374 gives us one single big cluster. Our goal is to divide as much of the data into moderate number of clusters. If we can approach our goal to divide them into two reasonable clusters and fit into (popular, non-popular), it would be best.  
<br>We choose eps = 0.374, then try different min_samples

In [122]:
# use disimatrix (distance) computed by nominal
dbscan = DBSCAN(eps=0.374, min_samples=8, metric="precomputed").fit_predict(dismatrix)
print('min_samples=8\n', dbscan[:100])
print('number of clusters: ', max(dbscan))
print('number of noises: ', len([x for x in dbscan if x == -1]))

dbscan = DBSCAN(eps=0.374, min_samples=9, metric="precomputed").fit_predict(dismatrix)
print('min_samples=9\n', dbscan[:100])
print('number of clusters: ', max(dbscan))
print('number of noises: ', len([x for x in dbscan if x == -1]))

dbscan = DBSCAN(eps=0.374, min_samples=13, metric="precomputed").fit_predict(dismatrix)
print('min_samples=10\n', dbscan[:100])
print('number of clusters: ', max(dbscan))
print('number of noises: ', len([x for x in dbscan if x == -1]))

min_samples=8
 [-1  1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1
 -1 -1 -1 -1  2 -1 -1  0 -1 -1  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
number of clusters:  14
number of noises:  3900
min_samples=9
 [-1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1
 -1 -1 -1 -1  1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
number of clusters:  12
number of noises:  4015
min_samples=10
 [-1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1

As shown above, no matter which set of parameter we chose, clusering using DBSCAN gives us <b>too many noise points</b>.

### Conclusion:
<br>DBSCAN is not a good method to cluster our dataset. The reason might be that the features in our dataset are mostly categorical instead of numerical. And when computing similarity, SMC is used. The dissimilarity is used as disttance. However, it is not strictly a metric because it does not satisfy <b>Triangle Inequality</b>.
<br>The dataset have both high density and low density. The rather low density data are all mistaken as noise points.
<br>To sum up, DBSCAN, a clustering method used to separate clusters of high density from noise points, is not suitable for our data.

## 2. CURE